## DSM

In [7]:
import pandas as pd
import ragraph.plot
import ragraph.analysis.sequence as seq
import ragraph.analysis.cluster as cluster
from ragraph.io.csv import from_csv

# --- 1. Load graph from your CSVs ---
# First, we identify the 'criticality' column dynamically (last column of edges.csv)
edges_df_temp = pd.read_csv("edges.csv", sep=";")
criticality_col = edges_df_temp.columns[-1]
print(f"Using '{criticality_col}' as weight for sequencing and clustering.")

g = from_csv(
    nodes_path="nodes.csv",
    edges_path="edges.csv",
    csv_delimiter=";",
    iter_delimiter=","
)

# --- 2. Apply Analysis (Clustering & Sequencing) ---
# A. Clustering: Group nodes into modules based on flow (using criticality)
cluster.hierarchical_markov(g, weights=criticality_col)

# B. Sequencing: Reorder nodes to minimize feedback loops (using criticality)
# This reorders g.nodes in-place.
seq.genetic(g, weights=criticality_col)

# --- 3. Build MDM ordering (Hardware -> Software -> Process) ---
# We want to enforce the Domain order but keep the intelligent sequencing
# we just calculated WITHIN those domains.

# 1. Define the Domain Priority (Hardware first = 0)
domain_priority = {"Hardware": 0, "Software": 1, "Process": 2}

# 2. Map nodes to their domain
nodes_df = pd.read_csv("nodes.csv", sep=";")
kind_map = dict(zip(nodes_df['name'], nodes_df['kind']))

# 3. Create the final ordered list of leaf objects
# We take the ALREADY SEQUENCED nodes from 'g' (g.nodes) and perform a stable sort by domain.
# This keeps the genetic sequence intact within the "Software" block, etc.
ordered_leafs = sorted(
    g.nodes, 
    key=lambda n: domain_priority.get(kind_map.get(n.name, "Unknown"), 99)
)

# --- 4. Plot MDM ---
mdm = ragraph.plot.mdm(
    leafs=ordered_leafs,
    edges=g.edges,
    style=ragraph.plot.Style(
        piemap=dict(
            display="labels",
            mode="relative",
            fields=g.edge_labels 
        )
    )
)

# Output the plot object
mdm

Using 'motivation' as weight for sequencing and clustering.
